In [1]:
import pandas as pd
import numpy as np
import OANDA_api as api
import Indicators

In [2]:
sess = api.oanda_api()

In [3]:
candles = sess.complete_candles_df('USD_CAD', 35000, 'H2', 'MBA')

In [4]:
candles.head()

,time,volume,mid_o,mid_h,mid_l,mid_c,bid_o,bid_h,bid_l,bid_c,ask_o,ask_h,ask_l,ask_c
0,2011-05-20T17:00:00.000000000Z,2728,0.97194,0.97232,0.96950,0.97224,0.97183,0.97221,0.96939,0.97212,0.97206,0.97244,0.96962,0.97235
1,2011-05-20T19:00:00.000000000Z,1758,0.97226,0.97412,0.97197,0.97404,0.97214,0.97362,0.97151,0.97354,0.97237,0.97462,0.97209,0.97454
2,2011-05-20T21:00:00.000000000Z,1,0.97404,0.97404,0.97404,0.97404,0.97354,0.97354,0.97354,0.97354,0.97454,0.97454,0.97454,0.97454
3,2011-05-20T23:00:00.000000000Z,1,0.97404,0.97404,0.97404,0.97404,0.97354,0.97354,0.97354,0.97354,0.97454,0.97454,0.97454,0.97454
4,2011-05-21T01:00:00.000000000Z,1,0.97404,0.97404,0.97404,0.97404,0.97354,0.97354,0.97354,0.97354,0.97454,0.97454,0.97454,0.97454


In [5]:
Indicators.relative_strength_index(candles)
Indicators.macd(candles)
Indicators.stochastic(candles)

/Users/ronaldsmith/PycharmProjects/FXbot/Indicators.py:114: RuntimeWarning: divide by zero encountered in double_scalars
  rs.append(100 - (100 / (1 + (cag / cal))))
/Users/ronaldsmith/PycharmProjects/FXbot/Indicators.py:120: RuntimeWarning: divide by zero encountered in double_scalars
  rs.append(100 - (100 / (1 + (cag / (cal)))))
/Users/ronaldsmith/PycharmProjects/FXbot/Indicators.py:154: RuntimeWarning: invalid value encountered in double_scalars
  stochastic_k = ((curr_close - low) / (high - low)) * 100


In [6]:
candles['spread'] = candles['ask_o'] - candles['bid_o']

In [7]:
candles['mid_c_prev'] = candles['mid_c'].rolling(1).mean()

In [8]:
candles['long'] = False
candles['short'] = False

for candle in range(len(candles)):
    if (candles.loc[candle, 'bid_o'] + 0.001 <= candles.loc[candle, 'ask_h'] and 
        candles.loc[candle, 'bid_o'] - 0.003 <= candles.loc[candle, 'ask_l']):
        candles.loc[candle, 'long'] = True
    if (candles.loc[candle, 'ask_o'] - 0.001 >= candles.loc[candle, 'bid_l'] and 
        candles.loc[candle, 'ask_o'] + 0.003 >= candles.loc[candle, 'bid_h']):
        candles.loc[candle, 'short'] = True

In [9]:
candles['rsi_14_lag_1'] = candles['rsi_14'].rolling(1).mean()
candles['MACD_12_26_9_lag_1'] = candles['MACD_12_26_9'].rolling(1).mean()

for lag in range(2,11):
    candles[f'rsi_14_lag_{lag}'] = candles[f'rsi_14_lag_{lag-1}'].rolling(1).mean()
    candles[f'MACD_12_26_9_lag_{lag}'] = candles[f'MACD_12_26_9_lag_{lag-1}'].rolling(1).mean()

In [10]:
candles['mid_o_lag_1'] = candles['mid_o'].rolling(1).mean()
candles['mid_l_lag_1'] = candles['mid_l'].rolling(1).mean()
candles['mid_h_lag_1'] = candles['mid_h'].rolling(1).mean()

for lag in range(2,11):
    candles[f'mid_o_lag_{lag}'] = candles[f'mid_o_lag_{lag-1}'].rolling(1).mean()
    candles[f'mid_l_lag_{lag}'] = candles[f'mid_l_lag_{lag-1}'].rolling(1).mean()
    candles[f'mid_h_lag_{lag}'] = candles[f'mid_h_lag_{lag-1}'].rolling(1).mean()

In [11]:
candles['range_14'] = float('nan') 

for candle in range(14,len(candles)):
    low = min(candles.loc[candle-14 : candle, f'mid_l'])
    high = max(candles.loc[candle-14 : candle, f'mid_h'])
    candles.loc[candle, 'range_14'] = high - low

In [12]:
candles['stochastic_range_k'] = candles['range_14'] * candles['stochastic_k']
candles['stochastic_range_d'] = candles['range_14'] * candles['stochastic_d']

In [13]:
candles['stochastic_range_k_lag_1'] = candles['stochastic_range_k'].rolling(1).mean()
candles['stochastic_range_d_lag_1'] = candles['stochastic_range_d'].rolling(1).mean()

for lag in range(2,11):
    candles[f'stochastic_range_k_lag_{lag}'] = candles['stochastic_range_k'].rolling(1).mean()
    candles[f'stochastic_range_d_lag_{lag}'] = candles['stochastic_range_d'].rolling(1).mean()

In [14]:
candles.columns

Index(['time', 'volume', 'mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h',
       'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 'rsi_14',
       'MACD_12_26_9', 'stochastic_k', 'stochastic_d', 'spread', 'mid_c_prev',
       'long', 'short', 'rsi_14_lag_1', 'MACD_12_26_9_lag_1', 'rsi_14_lag_2',
       'MACD_12_26_9_lag_2', 'rsi_14_lag_3', 'MACD_12_26_9_lag_3',
       'rsi_14_lag_4', 'MACD_12_26_9_lag_4', 'rsi_14_lag_5',
       'MACD_12_26_9_lag_5', 'rsi_14_lag_6', 'MACD_12_26_9_lag_6',
       'rsi_14_lag_7', 'MACD_12_26_9_lag_7', 'rsi_14_lag_8',
       'MACD_12_26_9_lag_8', 'rsi_14_lag_9', 'MACD_12_26_9_lag_9',
       'rsi_14_lag_10', 'MACD_12_26_9_lag_10', 'mid_o_lag_1', 'mid_l_lag_1',
       'mid_h_lag_1', 'mid_o_lag_2', 'mid_l_lag_2', 'mid_h_lag_2',
       'mid_o_lag_3', 'mid_l_lag_3', 'mid_h_lag_3', 'mid_o_lag_4',
       'mid_l_lag_4', 'mid_h_lag_4', 'mid_o_lag_5', 'mid_l_lag_5',
       'mid_h_lag_5', 'mid_o_lag_6', 'mid_l_lag_6', 'mid_h_lag_6',
       'mid_o_lag_7', 'mi

In [15]:
candle_data = ['mid_o_lag_1', 'mid_l_lag_1', 'mid_h_lag_1',
       'mid_o_lag_2', 'mid_l_lag_2', 'mid_h_lag_2', 'mid_o_lag_3',
       'mid_l_lag_3', 'mid_h_lag_3', 'mid_o_lag_4', 'mid_l_lag_4',
       'mid_h_lag_4', 'mid_o_lag_5', 'mid_l_lag_5', 'mid_h_lag_5',
       'mid_o_lag_6', 'mid_l_lag_6', 'mid_h_lag_6', 'mid_o_lag_7',
       'mid_l_lag_7', 'mid_h_lag_7', 'mid_o_lag_8', 'mid_l_lag_8',
       'mid_h_lag_8', 'mid_o_lag_9', 'mid_l_lag_9', 'mid_h_lag_9']

for col in candle_data:
    candles[col] = candles['mid_o'] - candles[col]

In [16]:
candles.drop(['mid_o', 'mid_h', 'mid_l', 'mid_c', 'bid_o', 'bid_h', 
              'bid_l', 'bid_c', 'ask_o', 'ask_h', 'ask_l', 'ask_c', 
              'time', 'volume'],
             axis=1, inplace=True)

In [17]:
candles.dropna(inplace=True)

In [19]:
candles.to_pickle('CAD_USD_H2')